In [ ]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.5 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import sqlite3
import os
from datetime import datetime

# =========================
# CSS CUSTOM (LIGHT / DARK)
# =========================
if st.session_state.get("dark_mode", False):
    st.markdown("""
    <style>
    .stApp { background-color: #121212; color: #e0e0e0; }

    h1, h2, h3, h4 { color: #90caf9; }

    section[data-testid="stSidebar"] {
        background-color: #1e1e1e;
        border-right: 1px solid #333;
    }

    .card {
        background-color: #1e1e1e;
        padding: 20px;
        border-radius: 12px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.6);
        margin-bottom: 20px;
    }

    .stButton>button {
        background-color: #90caf9;
        color: #000;
        border-radius: 8px;
        border: none;
    }

    [data-testid="stMetric"] {
        background-color: #1e1e1e;
        padding: 15px;
        border-radius: 10px;
    }
    </style>
    """, unsafe_allow_html=True)

else:
    st.markdown("""
    <style>
    .stApp { background-color: #f8f9fa; }

    h1, h2, h3, h4 { color: #0d6efd; }

    section[data-testid="stSidebar"] {
        background-color: #black;
        border-right: 1px solid #dee2e6;
    }


    .card {
        background-color: darkblue;
        padding: 20px;
        border-radius: 12px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.05);
        margin-bottom: 20px;
    }

    .stButton>button {
        background-color: #0d6efd;
        color: darkblue;
        border-radius: 8px;
        border: none;
    }

    [data-testid="stMetric"] {
        background-color: darkblue;
        padding: 15px;
        border-radius: 10px;
    }
    </style>
    """, unsafe_allow_html=True)

# =========================
# KONFIGURASI HALAMAN
# =========================
st.set_page_config(
    page_title="Mading Kelas SI",
    page_icon="📌",
    layout="centered"
)

# =========================
# DATABASE & FOLDER
# =========================
DB_FILE = "mading.db"
IMAGE_DIR = "images"
os.makedirs(IMAGE_DIR, exist_ok=True)

conn = sqlite3.connect(DB_FILE, check_same_thread=False)
cursor = conn.cursor()

# =========================
# BUAT TABEL (JIKA BELUM ADA)
# =========================
cursor.execute("""
CREATE TABLE IF NOT EXISTS postingan (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    judul TEXT,
    isi TEXT,
    penulis TEXT,
    tanggal TEXT,
    gambar TEXT
)
""")
conn.commit()

# =========================
# LOGIN ADMIN
# =========================
if "login" not in st.session_state:
    st.session_state.login = False

ADMIN_USER = "admin"
ADMIN_PASS = "admin123"

st.sidebar.title("🔐 Login Admin")

if not st.session_state.login:
    u = st.sidebar.text_input("Username")
    p = st.sidebar.text_input("Password", type="password")
    if st.sidebar.button("Login"):
        if u == ADMIN_USER and p == ADMIN_PASS:
            st.session_state.login = True
            st.rerun()
        else:
            st.sidebar.error("Login gagal")
else:
    st.sidebar.success("Login sebagai Admin")
    if st.sidebar.button("Logout"):
        st.session_state.login = False
        st.rerun()
# =========================
# DARK MODE TOGGLE
# =========================
if "dark_mode" not in st.session_state:
    st.session_state.dark_mode = False

st.sidebar.markdown("### 🎨 Tampilan")
st.session_state.dark_mode = st.sidebar.checkbox(
    "Aktifkan Dark Mode",
    value=st.session_state.dark_mode
)


# =========================
# NAVIGASI
# =========================
st.sidebar.markdown("---")
menu_items = ["🏠 Beranda", "📢 Pengumuman"]

if st.session_state.login:
    menu_items.insert(1, "➕ Buat Postingan")
    menu_items.insert(2, "✏️ Kelola Postingan")

menu = st.sidebar.radio("Navigasi", menu_items)

# =========================
# HEADER
# =========================
st.markdown(
    "<h1 style='text-align:center'>Mading Digital Kelas Sistem Informasi</h1><hr>",
    unsafe_allow_html=True
)

# =========================
# BERANDA
# =========================
mode = "🌙 Dark Mode" if st.session_state.dark_mode else "☀️ Light Mode"
st.caption(f"Tampilan saat ini: {mode}")

if menu == "🏠 Beranda":
    total = cursor.execute("SELECT COUNT(*) FROM postingan").fetchone()[0]
    st.metric("📢 Total Postingan", total)
    st.write("Sistem informasi mading digital berbasis SQLite.")

# =========================
# TAMBAH POSTINGAN
# =========================
elif menu == "➕ Buat Postingan":
    st.subheader("➕ Buat Postingan")

    with st.form("tambah"):
        judul = st.text_input("Judul")
        isi = st.text_area("Isi")
        penulis = st.text_input("Penulis")
        gambar = st.file_uploader("Upload Gambar", type=["jpg","png","jpeg"])
        simpan = st.form_submit_button("Simpan")

    if simpan and judul and isi and penulis:
        nama_gambar = ""
        if gambar:
            nama_gambar = datetime.now().strftime("%Y%m%d%H%M%S_") + gambar.name
            with open(os.path.join(IMAGE_DIR, nama_gambar), "wb") as f:
                f.write(gambar.getbuffer())

        cursor.execute("""
            INSERT INTO postingan (judul, isi, penulis, tanggal, gambar)
            VALUES (?, ?, ?, ?, ?)
        """, (
            judul, isi, penulis,
            datetime.now().strftime("%d-%m-%Y %H:%M"),
            nama_gambar
        ))
        conn.commit()
        st.success("Postingan berhasil ditambahkan")

# =========================
# EDIT & HAPUS POSTINGAN
# =========================
elif menu == "✏️ Kelola Postingan":
    st.subheader("✏️ Kelola Postingan")

    data = cursor.execute("SELECT * FROM postingan").fetchall()

    if not data:
        st.info("Belum ada postingan")
    else:
        pilihan = st.selectbox(
            "Pilih Postingan",
            data,
            format_func=lambda x: x[1]
        )

        id_post = pilihan[0]

        judul = st.text_input("Judul", pilihan[1])
        isi = st.text_area("Isi", pilihan[2])
        penulis = st.text_input("Penulis", pilihan[3])

        col1, col2 = st.columns(2)

        with col1:
            if st.button("💾 Simpan Perubahan"):
                cursor.execute("""
                    UPDATE postingan
                    SET judul=?, isi=?, penulis=?
                    WHERE id=?
                """, (judul, isi, penulis, id_post))
                conn.commit()
                st.success("Postingan diperbarui")
                st.rerun()

        with col2:
            if st.button("🗑️ Hapus Postingan"):
                if pilihan[5]:
                    img = os.path.join(IMAGE_DIR, pilihan[5])
                    if os.path.exists(img):
                        os.remove(img)

                cursor.execute("DELETE FROM postingan WHERE id=?", (id_post,))
                conn.commit()
                st.warning("Postingan dihapus")
                st.rerun()

# =========================
# TAMPILKAN POSTINGAN
# =========================
elif menu == "📢 Pengumuman":
    data = cursor.execute(
        "SELECT * FROM postingan ORDER BY id DESC"
    ).fetchall()

    if not data:
        st.info("Belum ada postingan")
    else:
        for d in data:
            st.markdown(f"## {d[1]}")
            st.caption(f"{d[3]} | {d[4]}")
            if d[5]:
                st.image(os.path.join(IMAGE_DIR, d[5]), use_container_width=True)
            st.write(d[2])
            st.markdown("---")


Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("36v3P2f3IUbgpzHrKcUerPai0V8_7pi6tQfjfAa61CxB1Fvq8")

public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://dorthy-prorealistic-monographically.ngrok-free.dev" -> "http://localhost:8501">